<a href="https://colab.research.google.com/github/Teasotea/BioNER-and-RD/blob/main/RD_KnowledgeGraphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import io

In [5]:
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import statistics as stats

In [1]:
import nltk
from nltk.tokenize import TreebankWordTokenizer as twt
import spacy
from spacy import displacy

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string 

In [2]:
import pickle
from google.colab import files

In [84]:
import bs4
import requests
nlp = spacy.load('en_core_web_sm')
nltk.download('stopwords')
nltk.download('punkt')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

fatal: destination path 'bert-sklearn' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-py3-none-any.whl size=54247 sha256=561a7898f0bc1720b8547599054e02641018ad2e26bcdcdb7d8ec0d06f10e7d1
  Stored in directory: /root/.cache/pip/wheels/e7/d4/73/12b2219a5cd4cd8c7acfd72204603ce34b6d2e4f620b205a80
Successfully built bert-sklearn
  Attempting uninstall: bert-sklearn
    Found existing installation: bert-sklearn 0.3

In [9]:
# !pip install --upgrade sagemaker

In [8]:
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

In [8]:
# !nvidia-smi

In [17]:
import torch
print('pytorch version:', torch.__version__)
# print('GPU:',torch.cuda.get_device_name(0))

pytorch version: 1.11.0+cu113


In [10]:
flatten = lambda l: [item for sublist in l for item in sublist]

# Load Data

In [11]:
with open('/content/data0.txt') as f:
      eval_article = f.read()

In [14]:
data = pd.DataFrame()
data['paragraphs'] = eval_article.split('\n')
data['tokenized'] = data['paragraphs'].apply(lambda x:list(twt().tokenize(x)))
data.head()

,paragraphs,tokenized
0,Transductional analysis was applied to the Pseudomonas aeruginosa mutant PAO14 (hnc-1). This mutant can utilize L-histidinol as sole source of carbon and nitrogen and has a 60-fold increased histi...,"[Transductional, analysis, was, applied, to, the, Pseudomonas, aeruginosa, mutant, PAO14, (, hnc-1, ), ., This, mutant, can, utilize, L-histidinol, as, sole, source, of, carbon, and, nitrogen, and..."
1,"Bacteriocins produced by six strains of Rhizobium trifolii were found to be of the relatively low molecular weight, non-phage type. The molecular weights ranged from approximately 1-8 X 105 to 2-0...","[Bacteriocins, produced, by, six, strains, of, Rhizobium, trifolii, were, found, to, be, of, the, relatively, low, molecular, weight, ,, non-phage, type., The, molecular, weights, ranged, from, ap..."
2,The gamma haemolysin of Staphylococcus aureus \'Smith 5R\' was produced on Dolman-Wilson agar overlain with cellophane. Maximal yields of crude lysin with titres of 2000 to 4000 haemolytic units/m...,"[The, gamma, haemolysin, of, Staphylococcus, aureus, \'Smith, 5R\, ', was, produced, on, Dolman-Wilson, agar, overlain, with, cellophane., Maximal, yields, of, crude, lysin, with, titres, of, 2000..."
3,"Arginine and methionine transport by Aspergillus nidulans mycelium was investigated. A single uptake system is responsible for the transport of arginine, lysine and ornithine. Transport is energy-...","[Arginine, and, methionine, transport, by, Aspergillus, nidulans, mycelium, was, investigated., A, single, uptake, system, is, responsible, for, the, transport, of, arginine, ,, lysine, and, ornit..."
4,"Aphids transmitted poly-L-ornithine (PLO)-treated tobacco mosaic virus (TMV) when given acquistion and inoculation access periods as brief as 30 s and 2 min, respectively; the ability to transmit ...","[Aphids, transmitted, poly-L-ornithine, (, PLO, ), -treated, tobacco, mosaic, virus, (, TMV, ), when, given, acquistion, and, inoculation, access, periods, as, brief, as, 30, s, and, 2, min, ,, re..."


# Load NER Model

In [29]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

scibert_model = CPU_Unpickler(open('/content/scibert_model.pkl', 'rb')).load() #for cpu

In [30]:

#pickled_model =  pickle.load(open('/content/scibert_model.pkl', 'rb')) #for gpu

# Extract Named Entities 

In [32]:
data_named_entities = scibert_model.predict(data['tokenized'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/7 [00:00<?, ?it/s]

In [34]:
print(data_named_entities[4])

['O', 'O', 'B-Chemical', 'O', 'B-Chemical', 'O', 'O', 'B-Disease', 'I-Disease', 'I-Disease', 'O', 'B-Disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'B-Disease', 'O', 'O', 'O', 'B-Disease', 'I-Disease', 'O', 'B-Chemical', 'B-Disease', 'O', 'O', 'O', 'O'

In [159]:
def extract_entities_from_string(string):
  tokenized = list(twt().tokenize(string))
  preds = scibert_model.predict(pd.Series([tokenized]))[0]
  entities = []
  for i, j in zip(tokenized, preds):
    if j!= 'O':
      entities.append(i)
  return list(set(entities))

In [160]:
extract_entities_from_string(data['paragraphs'].iloc[4])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

['acid',
 'virus',
 'polyamino',
 'PLO-treated',
 'potato',
 'KCl',
 'poly-L-lysine',
 'mosaic',
 'poly-L-ornithine',
 'tobacco',
 'rattle',
 'TMV',
 'PLO']

Copied

In [82]:
def isNotStopWord(word):
    return word not in stopwords.words('english')

def preprocess(sent):
  sent = re.sub("[\(\[].*?[\)\]]", "", sent)
  tokens = []
  temp = ""
  words = word_tokenize(sent)
  # Removing punctuations except '<.>/<?>/<!>'
  punctuations = '"#$%&\'()*+,-/:;<=>@\\^_`{|}~'
  words = map(lambda x: x.translate(str.maketrans('','',punctuations)), words)
  
  #Now, we remove the stopwords
  words = map(str.lower,words)
  words = filter(lambda x: isNotStopWord(x),words)
  tokens = tokens + list(words)
  temp = ' '.join(word for word in tokens)
  return temp

In [86]:
data['preprocessed'] = data['paragraphs'].apply(lambda x: preprocess(x))

In [87]:
def get_entity(sent):
  ent1 = "" #subject entity
  ent2 = "" #object entity

  prev_token_text = "" #text from previous token
  prev_token_dep = ""  #depedency from previous token

  prefix = ""
  modifier = ""

  for tok in nlp(sent):
    #Go in only if it is not a punctuation, else next word
    if tok.dep_ != "punct":
      #Check if token is a compund word
      if tok.dep_ == "compound":
        prefix = tok.text
        #Check if previous token is also a compound
        if prev_token_dep == "compound":
          prefix = prev_token_text + " "+ prefix
      #Check if token is a modifier or not
      if tok.dep_.endswith("mod")==True:
        modifier = tok.text 
        #Check if previous token is a compound
        if prev_token_dep == "compound":
          modifier = prev_token_text + " " + modifier

      #Checking if subject
      if tok.dep_.find("subj") == True:
        ent1 =modifier+ " " + prefix + " " + tok.text
        prefix = ""
        modifer = ""
      #Checking if object
      if tok.dep_.find("obj") == True:
        ent2 =modifier+ " " + prefix + " " + tok.text
        prefix = ""
        modifer = ""

      #Update variables
      prev_token_text = tok.text
      prev_token_dep = tok.dep_

  return [ent1.strip(), ent2.strip()]

In [88]:
[ent1,ent2] = get_entity("Transductional analysis was applied to the Pseudomonas aeruginosa mutant PAO14 (hnc-1).")
print("Subj : {a}, obj : {b}".format(a = ent1, b = ent2))
get_entity("Transductional analysis was applied to the Pseudomonas aeruginosa mutant PAO14 (hnc-1).")

Subj : Transductional  analysis, obj : Transductional aeruginosa mutant PAO14


['Transductional  analysis', 'Transductional aeruginosa mutant PAO14']

In [89]:
data['entity_pairs']= data['preprocessed'].apply(lambda x: [get_entity(i) for i in x.split('. ')])

In [90]:
def get_relation(sent):
  matcher = Matcher(nlp.vocab)
  patterns = [[{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}]]

  matcher.add("matching_1", patterns) 
  doc = nlp(sent)
  matches = matcher(doc)
  if len(matches)>0:
    k = len(matches) - 1
    span = doc[matches[k][1]:matches[k][2]] 
  else: return None
  return(span.text)

In [91]:
get_entity("Transductional analysis was applied to the Pseudomonas aeruginosa mutant PAO14 (hnc-1)."), get_relation("Transductional analysis was applied to the Pseudomonas aeruginosa mutant PAO14 (hnc-1).")

(['Transductional  analysis', 'Transductional aeruginosa mutant PAO14'],
 'applied to')

In [92]:
data['relations'] = data['preprocessed'].apply(lambda x: [get_relation(i) for i in x.split('. ')])

In [93]:
entity_pairs = flatten(data['entity_pairs'].tolist())
relations = flatten(data['relations'].tolist())

In [94]:
subjects = [x[0] for x in entity_pairs]
objects = [x[1] for x in entity_pairs]

In [106]:
pd.Series(subjects).value_counts()[:20]

                                          196
results                                     6
effects                                     3
transductional  analysis                    2
specific  activity                          2
adenyl cyclase activity                     2
antigenic  group                            2
intravenous kcn doxapram hydrochloride      2
findings                                    2
authors                                     2
cells                                       2
activity                                    2
enzymatic  activity                         2
level                                       2
receptors                                   2
data                                        2
values                                      2
pentagastrin                                2
premature  decline                          1
recovery function                           1
dtype: int64

In [97]:
entity_pairs2 = entity_pairs
relations2 = relations
#We keep relations only for those entities whose both source and target are present
entity_pairs3 = []
relations3 = []
for i in tqdm(range(len(entity_pairs2))):
  if entity_pairs2[i][0]!='' and entity_pairs2[i][1]!='':
    entity_pairs3.append(entity_pairs2[i])
    relations3.append(relations2[i])
tqdm._instances.clear()


100%|██████████| 824/824 [00:00<00:00, 380754.27it/s]


In [125]:
sample_sent = 'Australia, Arginine and methionine transport by Aspergillus nidulans mycelium was investigated. A single uptake system is responsible for the transport of arginine, lysine and ornithine'
nlp_ents = nlp(sample_sent).ents # tuple
[str(word) for word in nlp_ents] #list of words

['Australia', 'Arginine', 'Aspergillus']

In [3]:
extract_entities_from_string(sample_sent)

NameError: ignored

In [2]:
extract_entities_from_string(entity_pairs[0][1])

NameError: ignored

In [1]:
# Function works badly, slowly iterates though huge list. Needs to be fixed

# source,target,edge = [], [], []

# for i,j in enumerate(entity_pairs):
#   #Getting the named entities for source
#   str_source = extract_entities_from_string(entity_pairs[i][0])
#   doc_source = ' '.join(str_source)
#   #Getting the named entities for target
#   str_target = extract_entities_from_string(entity_pairs[i][1])
#   doc_target = ' '.join(str_target)
#   if doc_source != '' or doc_target != '':
#     edge.append(relations[i])
#     source.append(entity_pairs[i][0])
#     target.append(entity_pairs[i][1])

In [ ]:
print("###################  Most popular source entites    ###################### \n",pd.Series(source).value_counts()[:10])
print("###################  Most popular target entites    ###################### \n",pd.Series(target).value_counts()[:10])
print("###################  Most popular relations         ###################### \n",pd.Series(relations).value_counts()[:20])

[article 1](https://swayattadaw.medium.com/building-a-small-knowledge-graph-using-ner-296930592bcf), [article 2, more complex](https://towardsdatascience.com/building-a-biomedical-knowledge-graph-2930c7e05760)